In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
dev_no = torch.cuda.current_device()
print(dev_no)
dev_name = torch.cuda.get_device_name()
print(dev_name)

cuda
0
NVIDIA GeForce GTX 1650 Ti


In [3]:
TRAIN_SPLIT = 0.9
BATCH_SIZE = 16
NUM_WORKERS = 16
PATH = "data_2016_oco2/csv_folder/oco2_4thjan2016.csv"




class OCO2Dataset(Dataset):
    def __init__(self, path, split = "train"):
        self.df = pd.read_csv(path)
        self.length= len(self.df)
        if split == "train":
            self.df = self.df.iloc[0:int(TRAIN_SPLIT*self.length),:]
        else:
            self.df = self.df.iloc[int(TRAIN_SPLIT*self.length):self.length,:]
        self.length = len(self.df)
        self.df.drop(["dates","time","Unnamed: 0"],axis= 1,inplace=True)
        self.df.dropna(inplace= True)

    def __getitem__(self,index):
        x = np.array(self.df.loc[index,self.df.columns != "xco2"],dtype=np.float32)
        x = torch.from_numpy(x)
        y = np.array(self.df.loc[index,"xco2"],dtype=np.float32)
        y = torch.from_numpy(y)
        return x, y
    def __len__(self):    
        return self.length

In [4]:
train = OCO2Dataset(path=PATH)
test = OCO2Dataset(path=PATH,split = "test")
train_dl = DataLoader(train,shuffle=True,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS,pin_memory=True)
test_dl = DataLoader(test,shuffle=True,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS,pin_memory=True)
print(f"TRAIN_BATCHES: {len(train_dl)}\nTEST_BATCHES: {len(test_dl)}")

TRAIN_BATCHES: 9741
TEST_BATCHES: 1083


In [5]:
class Model(torch.nn.Module):
 
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(in_features = 5, out_features = 1) 
        #self.normalizer = torch.nn.functional.normalize(input)
        self.relu = nn.ReLU()
    def forward(self, x):
        #x = self.normalizer(x)
        y_pred = self.relu(self.linear(x))
        return y_pred

In [6]:
model = Model()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [7]:
EPOCH = 10
losses = []
for epoch in range(EPOCH):
    for batch in train_dl:
        x, y =   batch
        pred_y = model(x)
        loss = criterion(pred_y, y)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        print('epoch {}, loss {}'.format(epoch, loss.item()))

/home/nikhil/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0, loss 156521.609375
epoch 0, loss 159406.5
epoch 0, loss 160551.375
epoch 0, loss 159759.640625
epoch 0, loss 160306.953125
epoch 0, loss 160618.171875
epoch 0, loss 160586.3125
epoch 0, loss 160111.46875
epoch 0, loss 159779.046875
epoch 0, loss 158381.59375
epoch 0, loss 156537.375
epoch 0, loss 158183.59375
epoch 0, loss 157831.40625
epoch 0, loss 156395.84375
epoch 0, loss 159081.1875
epoch 0, loss 156802.65625
epoch 0, loss 158115.28125
epoch 0, loss 159627.9375
epoch 0, loss 159793.1875
epoch 0, loss 160659.3125
epoch 0, loss 156976.421875
epoch 0, loss 156985.671875
epoch 0, loss 154711.03125
epoch 0, loss 159987.890625
epoch 0, loss 159722.640625
epoch 0, loss 158043.375
epoch 0, loss 160091.90625
epoch 0, loss 157788.421875
epoch 0, loss 160033.203125
epoch 0, loss 159321.828125
epoch 0, loss 159717.3125
epoch 0, loss 158917.046875
epoch 0, loss 160534.109375
epoch 0, loss 158581.625
epoch 0, loss 160785.234375
epoch 0, loss 158998.859375
epoch 0, loss 159541.421875
ep

/home/nikhil/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1, loss 159846.0625
epoch 1, loss 160321.21875
epoch 1, loss 160533.25
epoch 1, loss 158413.859375
epoch 1, loss 159593.015625
epoch 1, loss 160426.78125
epoch 1, loss 155849.90625
epoch 1, loss 157383.71875
epoch 1, loss 158461.171875
epoch 1, loss 159276.8125
epoch 1, loss 159464.71875
epoch 1, loss 157704.8125
epoch 1, loss 159602.296875
epoch 1, loss 158693.25
epoch 1, loss 160822.109375
epoch 1, loss 158655.0625
epoch 1, loss 159960.109375
epoch 1, loss 159807.59375
epoch 1, loss 154443.125
epoch 1, loss 160014.375
epoch 1, loss 156969.65625
epoch 1, loss 160195.484375
epoch 1, loss 157721.0
epoch 1, loss 160261.125
epoch 1, loss 157632.75
epoch 1, loss 159829.203125
epoch 1, loss 158794.609375
epoch 1, loss 159978.59375
epoch 1, loss 160009.796875
epoch 1, loss 159875.640625
epoch 1, loss 159692.625
epoch 1, loss 161032.25
epoch 1, loss 158703.40625
epoch 1, loss 159702.09375
epoch 1, loss 159020.390625
epoch 1, loss 159249.015625
epoch 1, loss 159249.953125
epoch 1, loss 1

In [ ]:
plt.plot(losses)

In [ ]:
losses = []
for batch in train_dl:
    x, y =   batch
    pred_y = model(x)
    loss = criterion(pred_y, y)
    losses.append(loss.item())
plt.plot(losses)